# this code does stuff

In [1]:
import sys
import numpy as np
from scipy.stats import randint
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
#from keras.processing import sequence
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras import optimizers
#from keras.layers import embedding
from tensorflow.keras.models  import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.callbacks import EarlyStopping
#from keras.utils import np_utils
import itertools
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [3]:
## loading autospectrum file and creating input and output data

indata1 = sio.loadmat('logmldata500.mat')['pwwseq']
indata2 = np.swapaxes(indata1,0,2)
indata = np.swapaxes(indata2,0,1)
dfin, dfout = indata[:22][:][:], indata[22:][:][:]
dfin = np.swapaxes(dfin, 0,1)
dfout = np.swapaxes(dfout, 0,1)
print(dfin.shape)
ind = np.random.RandomState(2).permutation(range(len(dfin)))
dfin, dfout = dfin[ind][:][:], dfout[ind][:][:]
#print(dfin.shape, dfin[0][5][94],dfin1[0][518][4])


(87, 22, 500)


In [4]:
nk = 1
n = str(nk)

In [ ]:
## cross validation. sz1 decides the size of the trining sample (22 times 500/sz1). 

for train_index, test_index in KFold(n_splits=25).split(dfin):
    train_X, test_X = dfin[train_index][:][:], dfin[test_index][:][:]
    train_y, test_y = dfout[train_index][:][:], dfout[test_index][:][:]
    train_X = np.swapaxes(train_X, 0,1)
    train_y = np.swapaxes(train_y,  0,1)
    test_X = np.swapaxes(test_X, 0,1)
    test_y = np.swapaxes(test_y,  0,1)
    
    sz1 = 20
    k1 = np.int64(train_X.shape[2]/sz1)
    k2 = np.int64(test_X.shape[2]/sz1)
    
    ## k1 and k2 are number of frequency points per sample and sz1 is number of samples per dataset
    
    train_X = np.reshape(train_X,(train_X.shape[0],train_X.shape[1]*sz1, k1))
    train_y = np.reshape(train_y,(train_y.shape[0],train_y.shape[1]*sz1,k1))
    test_X = np.reshape(test_X,(test_X.shape[0],test_X.shape[1]*sz1,k2))
    test_y = np.reshape(test_y,(test_y.shape[0],test_y.shape[1]*sz1,k2))
    train_X = np.swapaxes(train_X, 0,1)
    train_y = np.swapaxes(train_y,  0,1)
    test_X = np.swapaxes(test_X, 0,1)
    test_y = np.swapaxes(test_y,  0,1)

    #train_X = np.reshape(train_X,(train_X.shape[0],train_X.shape[1],1))
    #test_X = np.reshape(test_X,(test_X.shape[0],test_X.shape[1],1))

    #train_X = np.reshape(train_X,(train_X.shape[0],train_X.shape[1],1))
    #test_X = np.reshape(test_X,(test_X.shape[0],test_X.shape[1],1))
    train_y = np.reshape(train_y,(train_y.shape[0],train_y.shape[2]))
    test_y = np.reshape(test_y,(test_y.shape[0],test_y.shape[2]))





    #print(indata[4][15][0],indata1[0][4][15],train_X2[4][5][492],train_X[2997][4],ind[5])
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    
    #best_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 100)
    
    best_stop = [
    tf.keras.callbacks.EarlyStopping(patience=100, monitor = 'val_loss', verbose = 1),
    tf.keras.callbacks.ModelCheckpoint(filepath='models/best_apt_1model' + n + '.h5', save_best_only = True),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]
    
    
    model = Sequential()
    model.add(LSTM(100,input_shape=(train_X.shape[1],train_X.shape[2])))
    #add(Activation('sigmoid'))
    model.add(Dropout(0.5))



    #model.add(Flatten())
    #model.add(Dense(5))
    model.add(Dense(500/sz1))
    #model.add(Activation('sigmoid'))
    epochs = 2000
    batch_size = sz1
    optimizers.Adam()
    opt1 = optimizers.Adam(lr = 0.00001)
    model.compile(loss='mean_squared_error', optimizer = opt1)
    #tf.optimizers.Adam(learning_rate=0.1)
    print(model.summary())
    pww = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data = (test_X,test_y), shuffle = 'False', callbacks = best_stop)
    model.load_weights('models/best_apt_1model'+n+'.h5')
    yhat = model .predict(test_X)
    testset = ind[test_index]
    sio.savemat('Saved_outfiles/2-dblvisfilt-goodsamps-kf25-pt25-'+n+'.mat',{'yhat':yhat, 'test_y' :test_y, 'testset' :testset})
    nk = nk+1
    n = str(nk)

(1660, 22, 25) (1660, 25) (80, 22, 25) (80, 25)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               50400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 25)                2525      
Total params: 52,925
Trainable params: 52,925
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2000
83/83 [==============================] - 2s 21ms/step - loss: 1.2745 - val_loss: 1.1435
Epoch 2/2000
83/83 [==============================] - 1s 18ms/step - loss: 1.1601 - val_loss: 1.0540
Epoch 3/2000
83/83 [==============================] - 1s 17ms/step - loss: 1.0734 - val_loss: 0.9689
Epoch 4/2000
83/83 [======

In [30]:
yhat = model.predict(test_X)